# Assignment 4: Convolutional Neural Networks (CNN) for FashionMNIST

**Name:** Frances Uy




**ICS 435 Machine Learning Fundamentals**

**Objective:** The goal of this assignment is to implement a Convolutional Neural Network (CNN)
to classify images from the FashionMNIST dataset. You will learn how to preprocess the data,
build and train a CNN, and evaluate its performance.



**Dataset:** FashionMNIST is a dataset of 60,000 training images and 10,000 test images, each of
size 28x28 grayscale, categorized into 10 different fashion classes (e.g., T-shirt, dress, sneaker,
etc.).

### Imports

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
#from tensorflow.keras.datasets import mnist

ModuleNotFoundError: No module named 'tensorflow'

### Part 1: Preprocessing

**Load the FashionMNIST dataset using torchvision.datasets (for PyTorch) or
tensorflow.keras.datasets (for TensorFlow)**